# HEPData script for SUS-21-008 to make .yaml files

Input root files from: https://pmeiring.web.cern.ch/pmeiring/EWK_SUSY_combination/20230301_SummariesApproval/

In [150]:
from hepdata_lib import Submission, Table, RootFileReader, Variable, Uncertainty
import ROOT, pickle
submission = Submission()
submission.read_abstract("abstract.txt")
submission.add_link("Webpage with all figures and tables", "https://cms-results.web.cern.ch/cms-results/public-results/publications/SUS-21-008/")
#submission.add_link("arXiv", "Fill later")
#submission.add_record_id(1111, "inspire")

In [151]:
# Extract TGraphErrors and save the variables in a pickle file
def extract_y_values_and_errors(file_path, is_symmetric=False):
    # Open the ROOT file
    tf = ROOT.TFile.Open(file_path)
    if not tf or tf.IsZombie():
        raise RuntimeError(f"Error: Could not open the ROOT file '{file_path}'.")

    # Create a list to hold the variables for each TGraphAsymmErrors
    variable_list = []

    # Loop over all objects in the ROOT file
    key_list = tf.GetListOfKeys()
    for key in key_list:
        obj = key.ReadObj()

        # Check if the object is a TGraphAsymmErrors
        if isinstance(obj, ROOT.TGraphAsymmErrors):
            # Extract the y-values and asymmetric errors
            y_values = [obj.GetY()[i] for i in range(obj.GetN())]
            errors_low = [obj.GetErrorYlow(i) for i in range(obj.GetN())]
            errors_high = [obj.GetErrorYhigh(i) for i in range(obj.GetN())]
            # low errors minus
            errors_low = [i * -1 for i in errors_low]
            
            var_name = obj.GetName()
            variable = hepdata_lib.Variable(var_name, values=y_values, is_independent=False, is_binned=False)
            unc = hepdata_lib.Uncertainty(label=var_name+f"Uncertainty", is_symmetric=False)
            
            uncertain = []
            #Create separate asymmetric uncertainties for each point
            for i, (low, high) in enumerate(zip(errors_low, errors_high)):
                unc.minus = low
                unc.plus = high
                uncertain.append((low,high))
            
            unc.values = uncertain
            variable.add_uncertainty(unc)
            # Add the variable to the list
            variable_list.append(variable)

    # Close the ROOT file
    tf.Close()

    return variable_list


In [152]:
variable_list = extract_y_values_and_errors("hists_fig_6-8.root")
# Save the variables to a file
output_file = "variables.pkl"
with open(output_file, 'wb') as f:
    pickle.dump(variable_list, f)

loaded_variables = None
with open(output_file, 'rb') as f:
    loaded_variables = pickle.load(f)


In [153]:
#The TH1Ds from input files are extracted out of TCanvas locally and hists are read here, adding the variables from TGraphErrors to the table

def getTH1Ds(inputFile, fig, location, prefix, varss, title, xBin_name, unit):
    table = Table(fig)
    table.description = title
    table.location = location
    table.keywords["cmenergies"] = ["13000"]
    table.keywords["observables"] = ["N"]
    table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
    reader = RootFileReader(inputFile)
    #Data = reader.read_hist_1d(prefix+"data")
    
    for var in varss:
        readHist = reader.read_hist_1d(prefix+var)
        varHist = Variable("Number of "+var, is_independent=False, is_binned=False, units="")
        varHist.values = readHist["y"]
        table.add_variable(varHist)
    
    
    def find_variable_by_name(variable_list, name):
        for variable in variable_list:
            if variable.name == name:
                return variable
        return None
    
    v1 = find_variable_by_name(variable_list, prefix+"background_errors")
    table.add_variable(v1)
    v2 = find_variable_by_name(variable_list, prefix+"data_graph")
    table.add_variable(v2)
    
    X = reader.read_hist_1d(prefix+varss[0])
    xBins = Variable(xBin_name, is_independent=True, is_binned=True, units = unit)
    xBins.values = X["x_edges"]
    table.add_variable(xBins)
    
    
    submission.add_table(table)

In [154]:
#Fig 6

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 6-a",
    location = "Data on page 9",
    prefix   = "SR_2l_ewk_dM20_low_",
    varss = ["prompt_wz","prompt_vv","Rares","FRfakes_data","prompt_dy","prompt_tt","background","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 6-b",
    location = "Data on page 9",
    prefix   = "SR_2l_ewk_dM20_med_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","prompt_dy","prompt_tt","background","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 6-c",
    location = "Data on page 9",
    prefix   = "SR_2l_ewk_dM20_high_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","prompt_dy","prompt_tt","background","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 6-d",
    location = "Data on page 9",
    prefix   = "SR_2l_ewk_dM20_ultra_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","prompt_dy","prompt_tt","background","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

In [155]:
#Fig 7

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 7-a",
    location = "Data on page 10",
    prefix   = "SR_3l_ewk_dM20_low_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","background","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 3l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 7-b",
    location = "Data on page 10",
    prefix   = "SR_3l_ewk_dM20_med_",
    varss = ["prompt_wz","prompt_vv","Rares","semidd_fakes","background","signal_TChiWZ_200_180_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 3l soft category",
    xBin_name = "M(ll)",
    unit = "GeV")

In [156]:
# Fig 8

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 8-a",
    location = "Data on page 11",
    prefix   = "MT2_NLSP125_dM10_low_",
    varss = ["prompt_wz","prompt_vv","Rares","prompt_dy","prompt_tt","FRfakes_data","background","signal_TSlepSlep_125_115_norm"],
    title    = "Postfit distribution of m_{T2}(ll) variable for low p_(T)^{miss} bins in 2l soft signal region",
    xBin_name = "M_{T2}(ll)-100",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 8-b",
    location = "Data on page 11",
    prefix   = "MT2_NLSP125_dM10_med_",
    varss = ["prompt_wz","prompt_vv","Rares","prompt_dy","prompt_tt","semidd_fakes","background","signal_TSlepSlep_125_115_norm"],
    title    = "Postfit distribution of m_{T2}(ll) variable for medium p_(T)^{miss} bins in 2l soft signal region",
    xBin_name = "M_{T2}(ll)-100",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 8-c",
    location = "Data on page 11",
    prefix   = "MT2_NLSP125_dM10_high_",
    varss = ["prompt_wz","prompt_vv","Rares","prompt_dy","prompt_tt","semidd_fakes","background","signal_TSlepSlep_125_115_norm"],
    title    = "Postfit distribution of m_{T2}(ll) variable for high p_(T)^{miss} bins in 2l soft signal region",
    xBin_name = "M_{T2}(ll)-100",
    unit = "GeV")

getTH1Ds(
    inputFile  = "hists_fig_6-8.root",
    fig = "Figure 8-d",
    location = "Data on page 11",
    prefix   = "MT2_NLSP125_dM10_ultra_",
    varss = ["prompt_wz","prompt_vv","Rares","prompt_dy","prompt_tt","semidd_fakes","background","signal_TSlepSlep_125_115_norm"],
    title    = "Postfit distribution of m_{T2}(ll) variable for ultra p_(T)^{miss} bins in 2l soft signal region",
    xBin_name = "M_{T2}(ll)-100",
    unit = "GeV")

In [157]:
def getTH1Ds_noAdding(inputFile, fig, location, prefix, varss, title, xBin_name, unit):
    table = Table(fig)
    table.description = title
    table.location = location
    table.keywords["cmenergies"] = ["13000"]
    table.keywords["observables"] = ["N"]
    table.keywords["phrases"] = ["Supersymmetry", "SUSY"]
    reader = RootFileReader(inputFile)
    
    for var in varss:
        readHist = reader.read_hist_1d(prefix+var)
        varHist = Variable("Number of "+var, is_independent=False, is_binned=False, units="")
        varHist.values = readHist["y"]
        if "total" in var:
            unc_totalbackground = Uncertainty("Total uncertainty", is_symmetric=True)
            unc_totalbackground.values = readHist["dy"]
            varHist.add_uncertainty(unc_totalbackground)
        if var == prefix+"_data":
            unc_data = Uncertainty("Data uncertainty", is_symmetric=True)
            unc_data.values = readHist["dy"]
            varHist.add_uncertainty(unc_data)
    
        table.add_variable(varHist)
        
    
    X = reader.read_hist_1d(prefix+varss[0])
    xBins = Variable(xBin_name, is_independent=True, is_binned=True, units= unit)
    xBins.values = X["x_edges"]
    table.add_variable(xBins)
    
    submission.add_table(table)

In [158]:
# Fig 9

getTH1Ds_noAdding(
    inputFile  = "figures_9-10.root",
    fig = "Figure 9",
    location = "Data on page 14",
    prefix   = "SR_3lAnew_",
    varss = ["data","prompt_WZ","prompt_VH","data_fakes","prompt_ZZ","prompt_TTX","total","TChiWZ_200_160_norm","TChiWZ_600_100_norm","TChiWH_150_1_norm","TChiWH_150_25_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 2l soft category",
    xBin_name = "Search bin number",
    unit = "GeV"
)

In [159]:
# Fig 10

getTH1Ds_noAdding(
    inputFile  = "plots_wz-2.root",
    fig = "Figure 10",
    location = "Data on page 14",
    prefix   = "SR_3lB_",
    varss = ["data","prompt_WZ","prompt_VH","data_fakes","prompt_ZZ","prompt_TTX","convs", "total","TChiWH_150_1_norm","TChiWH_150_25_norm"],
    title    = "Postfit distribution of observed and expected yields for signal regions in the 3l soft category",
    xBin_name = "min(dR(l,l))",
    unit = ""
)

In [160]:
def get2DScanFromText(txtfile, depUncs = [], pdf = None, qualifiers = [], obs = None, location = "", title = "", xlabel = "", ylabel = "", zlabel="", unitsx="GeV", unitsy="GeV", unitsz="", ttitle=""):
  print(txtfile)
  inPDF = pdf
  fig = Table(ttitle)
  fig.description = title
  fig.location    = location
  fig.keywords["observables"] = obs
  theDict = {"Mass1": 0, "Mass2": 1, "Expected, median":4, "Obsserved":7, "Expected, -2$\sigma$": 2, "Expected, -1$\sigma$":3, "Expected, +1$\sigma$": 5, "Expected, +2$\sigma$":6}  
  theVals = {i:[] for i in theDict.keys()}
  xvals = []
  yvals = []
  theFile = open(txtfile,"r")
  for line in theFile.readlines():
     newvals = [float(a.replace(" ", "")) for a in line.split(":")]
     if len(newvals) < 7 or any([nn > 999 for nn in newvals]): continue
     for key in theVals.keys():
        theVals[key].append(newvals[theDict[key]])
     xvals.append(newvals[theDict["Mass1"]])
     yvals.append(newvals[theDict["Mass2"]])

  ## X axis
  xvar = Variable(xlabel, is_independent=True, is_binned=False, units=unitsx)
  xvar.values = xvals
  fig.add_variable(xvar)
  yvar = Variable(ylabel, is_independent=True, is_binned=False, units=unitsy)
  yvar.values = yvals
  fig.add_variable(yvar)  
  zvars = []
  for key in theDict.keys():
       if "Mass" in key: continue 
       zvars.append(Variable(key + zlabel, is_independent=False, is_binned=False, units=unitsz))
       zvars[-1].values = theVals[key]

       for q in qualifiers:
         zvars[-1].add_qualifier(*q)
       fig.add_variable(zvars[-1])

  #fig.add_image(inPDF)
  submission.add_table(fig)


In [161]:
#Fig 11
get2DScanFromText("summary_obs_Pure_NPFull_asymp_MLNNSOSparametriczedgeWX_WZ",
 pdf = None,
 location = "Figure 11 (top left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "TChiWZ")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_MLWHWX_WH",
 pdf = None,
 location = "Figure 11 (bottom left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "TChiWH")

get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLSOSparametriczedgeWXWH_WHWZ0p50",
 pdf = None,
 location = "Figure 11 (bottom right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "TChiWZ+TChiWH")

summary_obs_Pure_NPFull_asymp_MLNNSOSparametriczedgeWX_WZ
summary_obs_Pure_NPFull_asymp_MLWHWX_WH
summary_obs_Mixed_NPFull_asymp_MLSOSparametriczedgeWXWH_WHWZ0p50


In [162]:
# Fig 12 WZ
get2DScanFromText("summary_obs_Pure_NPFull_asymp_MLNN_WZ",
 pdf = None,
 location = "Figure 12 (top left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "MLNN_WZ")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_SOSparametric_WZ",
 pdf = None,
 location = "Figure 12 (top right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "SOSparametric_WZ")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_zedge_WZ",
 pdf = None,
 location = "Figure 12 (bottom left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "zedge_WZ")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_WX_WZ",
 pdf = None,
 location = "Figure 12 (bottom right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "hadr.WX")

summary_obs_Pure_NPFull_asymp_MLNN_WZ
summary_obs_Pure_NPFull_asymp_SOSparametric_WZ
summary_obs_Pure_NPFull_asymp_zedge_WZ
summary_obs_Pure_NPFull_asymp_WX_WZ


In [163]:
# Fig 12 WH
get2DScanFromText("summary_obs_Pure_NPFull_asymp_ML_WH",
 pdf = None,
 location = "Figure 12 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "ML_WH")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_WH_WH",
 pdf = None,
 location = "Figure 12 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "WH_WH")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_WX_WH",
 pdf = None,
 location = "Figure 12 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Hadr_WH")

summary_obs_Pure_NPFull_asymp_ML_WH
summary_obs_Pure_NPFull_asymp_WH_WH
summary_obs_Pure_NPFull_asymp_WX_WH


In [164]:
# Fig 13
get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ1p00",
 pdf = None,
 location = "Figure 13 (top left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "ZZ")

get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p00",
 pdf = None,
 location = "Figure 13 (top right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "HH")

get2DScanFromText("summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p50",
 pdf = None,
 location = "Figure 13 (bottom)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "ZH")

summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ1p00
summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p00
summary_obs_Mixed_NPFull_asymp_MLzedge4b_HHHZZZ0p50


In [165]:
# Fig 14
get2DScanFromText("summary_byBR",
 pdf = None,
 location = "Figure 14-a",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "BR Combined limit")

get2DScanFromText("summary_byBR_ML",
 pdf = None,
 location = "Figure 14-b",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "ML Limit")

get2DScanFromText("summary_byBR_zedge",
 pdf = None,
 location = "Figure 14-c",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Zedge Limit")

get2DScanFromText("summary_byBR_4b",
 pdf = None,
 location = "Figure 14-d",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "4b Limit")

summary_byBR
summary_byBR_ML
summary_byBR_zedge
summary_byBR_4b


In [166]:
# Fig 15
get2DScanFromText("summary_obs_Pure_NPFull_asymp_WX4bMLWH_WWWHHHhino",
 pdf = None,
 location = "Figure 15",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Higgsino combined limit")

summary_obs_Pure_NPFull_asymp_WX4bMLWH_WWWHHHhino


In [167]:
# Fig 16
get2DScanFromText("summary_obs_Pure_NPFull_asymp_SOSparametric_slep",
 pdf = None,
 location = "Figure 16 (left)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "SOS Combined limit")

get2DScanFromText("summary_obs_Pure_NPFull_asymp_SOSparametriczedge_slep",
 pdf = None,
 location = "Figure 16 (right)",
 title = "Upper limits at the 95 \% CL for the TChiWZ model, normalized to expected cross section",
 xlabel = "$m_{\\tilde{\chi}_2^{0}} = m_{\\tilde{\chi}_1^{\pm}} $",
 ylabel = "$m_{\\tilde{\chi}_1^{0}}$",
 zlabel = " Upper limit",
 unitsx = "GeV",
 unitsy = "GeV",
 unitsz = "fb",
 obs    = ["UL"],
 ttitle = "Compressed SOS limit")

summary_obs_Pure_NPFull_asymp_SOSparametric_slep
summary_obs_Pure_NPFull_asymp_SOSparametriczedge_slep


In [168]:
submission.create_files("Test",remove_old=True)